In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [2]:
# 1. Charger les données
# Charger le fichier CSV dans un DataFrame pandas
bureau = pd.read_csv(r'C:\Users\jason\Desktop\Cours MS2D\Bejaoui\IA\Credit scoring project\bureau.csv')

In [3]:
# Afficher un aperçu des premières lignes du DataFrame pour vérifier le chargement correct des données
print(bureau.head())

   SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
0      215354       5714462        Closed      currency 1         -497   
1      215354       5714463        Active      currency 1         -208   
2      215354       5714464        Active      currency 1         -203   
3      215354       5714465        Active      currency 1         -203   
4      215354       5714466        Active      currency 1         -629   

   CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
0                   0               -153.0             -153.0   
1                   0               1075.0                NaN   
2                   0                528.0                NaN   
3                   0                  NaN                NaN   
4                   0               1197.0                NaN   

   AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
0                     NaN                   0         91323.0   
1                     NaN         

In [4]:
# 2. Sélectionner les colonnes pertinentes
# Spécifier les colonnes pertinentes à conserver dans le DataFrame
selected_columns_bureau = [
    'SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'DAYS_CREDIT', 
    'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 
    'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 
    'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 
    'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY'
]

# Créer un DataFrame contenant uniquement les colonnes sélectionnées
bureau_selected = bureau[selected_columns_bureau].copy()

In [5]:
# Afficher le nombre de valeurs manquantes par colonne
print(bureau_selected.isnull().sum())

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        105553
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_MAX_OVERDUE    1124488
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                 13
AMT_CREDIT_SUM_DEBT        257669
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
AMT_ANNUITY               1226791
dtype: int64


In [6]:
# Traiter les valeurs manquantes
bureau_selected.loc[:, 'DAYS_CREDIT_ENDDATE'] = bureau_selected['DAYS_CREDIT_ENDDATE'].fillna(bureau_selected['DAYS_CREDIT_ENDDATE'].median())
bureau_selected.loc[:, 'DAYS_ENDDATE_FACT'] = bureau_selected['DAYS_ENDDATE_FACT'].fillna(bureau_selected['DAYS_ENDDATE_FACT'].median())
bureau_selected.loc[:, 'AMT_CREDIT_MAX_OVERDUE'] = bureau_selected['AMT_CREDIT_MAX_OVERDUE'].fillna(0)
bureau_selected.loc[:, 'AMT_CREDIT_SUM'] = bureau_selected['AMT_CREDIT_SUM'].fillna(bureau_selected['AMT_CREDIT_SUM'].median())
bureau_selected.loc[:, 'AMT_CREDIT_SUM_DEBT'] = bureau_selected['AMT_CREDIT_SUM_DEBT'].fillna(bureau_selected['AMT_CREDIT_SUM_DEBT'].median())
bureau_selected.loc[:, 'AMT_CREDIT_SUM_LIMIT'] = bureau_selected['AMT_CREDIT_SUM_LIMIT'].fillna(bureau_selected['AMT_CREDIT_SUM_LIMIT'].median())
bureau_selected.loc[:, 'AMT_ANNUITY'] = bureau_selected['AMT_ANNUITY'].fillna(bureau_selected['AMT_ANNUITY'].median())

In [7]:
# Sélectionner les colonnes de type 'object' et obtenir le nombre de valeurs uniques
object_columns_unique_counts = bureau_selected.select_dtypes('object').apply(pd.Series.nunique, axis=0)
print(object_columns_unique_counts)

# Afficher les valeurs uniques pour chaque colonne de type 'object'
for column in bureau_selected.select_dtypes('object').columns:
    print(f"Valeurs uniques pour {column}:")
    print(bureau_selected[column].unique())
    print()

CREDIT_ACTIVE       4
CREDIT_CURRENCY     4
CREDIT_TYPE        15
dtype: int64
Valeurs uniques pour CREDIT_ACTIVE:
['Closed' 'Active' 'Sold' 'Bad debt']

Valeurs uniques pour CREDIT_CURRENCY:
['currency 1' 'currency 2' 'currency 4' 'currency 3']

Valeurs uniques pour CREDIT_TYPE:
['Consumer credit' 'Credit card' 'Mortgage' 'Car loan' 'Microloan'
 'Loan for working capital replenishment' 'Loan for business development'
 'Real estate loan' 'Unknown type of loan' 'Another type of loan'
 'Cash loan (non-earmarked)' 'Loan for the purchase of equipment'
 'Mobile operator loan' 'Interbank credit'
 'Loan for purchase of shares (margin lending)']



In [8]:
# 5. Transformation logarithmique pour les colonnes financières
# Appliquer une transformation logarithmique (log1p) sur certaines colonnes pour réduire l'impact des valeurs extrêmes
columns_to_log_transform = ['AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_ANNUITY']

for col in columns_to_log_transform:
    bureau_selected[col] = bureau_selected[col].apply(lambda x: np.log1p(x) if x > 0 else 0)

In [9]:
# 5. Imputation des valeurs aberrantes
numerical_cols_bureau = ['DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 
                         'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_ANNUITY']

In [10]:
# Vérifier les types de données
print(bureau_selected.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int64  
 1   SK_ID_BUREAU            int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int64  
 5   CREDIT_DAY_OVERDUE      int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int64  
 16  AMT_ANNUITY             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB
None


In [11]:
# Nombre de lignes avant le filtrage des valeurs aberrantes
print(f"Nombre de lignes avant le filtrage des valeurs aberrantes : {len(bureau_selected)}")

Nombre de lignes avant le filtrage des valeurs aberrantes : 1716428


In [12]:
# Imputation des valeurs en dehors des quantiles 0.01 et 0.99 par la médiane de chaque colonne
for col in numerical_cols_bureau:
    q_low = bureau_selected[col].quantile(0.01)
    q_high = bureau_selected[col].quantile(0.99)
    
    # Remplacer les valeurs en dehors des quantiles par la médiane
    median_value = bureau_selected[col].median()
    bureau_selected[col] = bureau_selected[col].mask((bureau_selected[col] < q_low) | (bureau_selected[col] > q_high), median_value)

In [13]:
# Réduire la taille des données avec un échantillonnage aléatoire
bureau_sampled = bureau_selected.sample(frac=0.1, random_state=42)  # 10% des données

# Appliquer l'Isolation Forest sur l'échantillon
iso = IsolationForest(contamination=0.01, random_state=42)
yhat = iso.fit_predict(bureau_sampled[numerical_cols_bureau])


In [14]:
# Filtrer les anomalies détectées dans l'échantillon
bureau_sampled = bureau_sampled[yhat != -1]

In [15]:
# Fusionner les résultats avec les données originales pour conserver les non-anomalies
bureau_selected = pd.concat([bureau_selected[~bureau_selected.index.isin(bureau_sampled.index)], bureau_sampled])

In [16]:
# Nombre de lignes après le filtrage des valeurs aberrantes
print(f"Nombre de lignes après le filtrage des valeurs aberrantes : {len(bureau_selected)}")

Nombre de lignes après le filtrage des valeurs aberrantes : 1716428


In [17]:
# Vérifier que toutes les valeurs manquantes ont été traitées
print(bureau_selected.isnull().sum())

SK_ID_CURR                0
SK_ID_BUREAU              0
CREDIT_ACTIVE             0
CREDIT_CURRENCY           0
DAYS_CREDIT               0
CREDIT_DAY_OVERDUE        0
DAYS_CREDIT_ENDDATE       0
DAYS_ENDDATE_FACT         0
AMT_CREDIT_MAX_OVERDUE    0
CNT_CREDIT_PROLONG        0
AMT_CREDIT_SUM            0
AMT_CREDIT_SUM_DEBT       0
AMT_CREDIT_SUM_LIMIT      0
AMT_CREDIT_SUM_OVERDUE    0
CREDIT_TYPE               0
DAYS_CREDIT_UPDATE        0
AMT_ANNUITY               0
dtype: int64


In [36]:
# Sauvegarder les données nettoyées
bureau_selected.to_csv(r'C:\Users\jason\Desktop\Cours MS2D\Bejaoui\IA\Credit scoring project\bureau_clean.csv', index=False)